# Sprint 13 - Proyecto

#### Cargar las librerías

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

# Metrics
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
    auc,
    f1_score,
    mean_squared_error
)

# Utilities
from sklearn.utils import shuffle


In [2]:
#Cargar los archivos

gold_recovery_train = pd.read_csv('data/gold_recovery_train.csv')
gold_recovery_test = pd.read_csv('data/gold_recovery_test.csv')
gold_recovery_full = pd.read_csv('data/gold_recovery_full.csv')

In [4]:
#Análisis exploratorio de datos

gold_recovery_train.info()
gold_recovery_train.info()
gold_recovery_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16860 entries, 0 to 16859
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                16860 non-null  object 
 1   final.output.concentrate_ag                         16788 non-null  float64
 2   final.output.concentrate_pb                         16788 non-null  float64
 3   final.output.concentrate_sol                        16490 non-null  float64
 4   final.output.concentrate_au                         16789 non-null  float64
 5   final.output.recovery                               15339 non-null  float64
 6   final.output.tail_ag                                16794 non-null  float64
 7   final.output.tail_pb                                16677 non-null  float64
 8   final.output.tail_sol                               16715 non-null  float64
